# Dataset for Aspect-Based Sentiment Analysis and Topic Modeling on Borobudur Temple and Prambanan Temple by Dian Arianto
https://github.com/dian9395/dataset-analisis-sentimen-berbasis-aspek-dan-pemodelan-topik

In [1]:
import pandas as pd
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from tqdm import tqdm
import nltk
import re
from sklearn.feature_extraction.text import CountVectorizer

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [2]:
df = pd.read_excel('dataset/reviews_borobudur_prambanan_TripAdvisor_GMaps_all_tesis.xlsx')
df_clean = df.dropna()
# print("\nDrop rows with any NaN values:")
df_clean

,id,lokasi,text,daya_tarik,amenitas,aksesibilitas,citra,harga,sdm
0,1.0,Candi Borobudur,peninggalan sejarah yang sudah berumur 1200 ta...,1,-,-,1,0,0
1,2.0,Candi Borobudur,Pertama kali bepergian selama masa pandemi. Ca...,-,1,-,1,-,1
2,3.0,Candi Borobudur,"Candi Borobudur di Magelang, Yogyakarta adalah...",1,-,-,1,-,-
3,4.0,Candi Borobudur,"Baru pertama kali kesini, pas sih kalau tempat...",1,-,-,-,-1,-
4,5.0,Candi Borobudur,"candi borobudur, tempat wisata ini sudah terke...",1,1,-,1,-,-
...,...,...,...,...,...,...,...,...,...
6738,6739.0,Candi Prambanan,The place is great and probably everything you...,1,-,-,-,-1,-
6739,6740.0,Candi Prambanan,Prambanan or Rara Jonggrang (Javanese: ꦫꦫꦗꦺꦴꦁꦒ...,0,-,-,-,-,-
6740,6741.0,Candi Prambanan,This temple is not worth the price. There’s no...,-1,-,-,-1,-1,-1
6741,6742.0,Candi Prambanan,Prambanan temple is the largest hindu temple o...,1,-,-,-,-,-


In [3]:
review = df_clean['text']

## Preprocessing
Emoji processing; case folding; menghapus username, angka, dan tanda baca; koreksi ejaan dan singkatan serta menghapus whitespace; penghapusan stopwords, dan stemming.

In [7]:
import re
# Daftar emoji berdasarkan sentimen
emoji_positif = ["😊", "😄", "♥", "😍", "😘", "😃", "😁", "😆", "😇", "☺"]
emoji_negatif = ["😢", "😠", "😡", "😭", "😱", "😨", "😫", "😩", "😖", "😔"]
emoji_netral = ["👍", "✨", "★", "█", "👌", "♫", "�", "©", "💬", "🔔"]

# Fungsi untuk mengganti emoji dengan label sentimen
def emotion(teks):
    for emoji in emoji_positif:
        teks = teks.replace(emoji, "positif")
    for emoji in emoji_negatif:
        teks = teks.replace(emoji, "negatif")
    for emoji in emoji_netral:
        teks = teks.replace(emoji, "netral")
    return teks

In [4]:
import gdown
# URL dari file Google Drive
stopwords_uci_link = 'https://drive.google.com/uc?id=1qqSuE1tuDCCfsdKgXMC3Nj0jda1LuWXB'
stopwords_nltk_link = 'https://drive.google.com/uc?id=1Ufurgc02rF2_yuFh8GNw1VOpkNbJLKWx'
stopwords_uci_output = 'stopwords_uci.txt'
stopwords_nltk_output = 'stopwords_nltk.txt'
# Mengunduh file
gdown.download(stopwords_uci_link, stopwords_uci_output, quiet=False)
gdown.download(stopwords_nltk_link, stopwords_nltk_output, quiet=False)
# Membaca konten file
with open(stopwords_uci_output, 'r', encoding='utf-8') as file:
    stopwords_uci = file.read()
with open(stopwords_nltk_output, 'r', encoding='utf-8') as file:
    stopwords_nltk = file.read()

Downloading...
From: https://drive.google.com/uc?id=1qqSuE1tuDCCfsdKgXMC3Nj0jda1LuWXB
To: e:\PROJECT\Python-Pro\sa-TripAdvisor\stopwords_uci.txt
100%|██████████| 6.91k/6.91k [00:00<00:00, 359kB/s]
Downloading...
From: https://drive.google.com/uc?id=1Ufurgc02rF2_yuFh8GNw1VOpkNbJLKWx
To: e:\PROJECT\Python-Pro\sa-TripAdvisor\stopwords_nltk.txt
100%|██████████| 7.39k/7.39k [00:00<00:00, 698kB/s]


In [11]:
def preprocess(text):
    # text preprocessing
    teks = emotion(text)
    teks = teks.lower()
    teks = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    stop_words = set(stopwords.words('english') + stopwords_uci.split('\n') + stopwords_nltk.split('\n'))
    tokens = word_tokenize(teks)
    filtered_tokens = [word for word in tokens if word.isalnum() and not any(char.isdigit() for char in word) and word not in stop_words]
    stemmer_factory = StemmerFactory()
    sastrawi_stemmer = stemmer_factory.create_stemmer()
    lemmatizer = nltk.WordNetLemmatizer()
    lemma = [lemmatizer.lemmatize(word) for word in filtered_tokens]
    indonesian_stemmed_words = [sastrawi_stemmer.stem(word) for word in lemma]
    clean_reviews = ' '.join(indonesian_stemmed_words)
    return clean_reviews

In [12]:
teks = review.apply(preprocess)
teks

0       tinggal sejarah umur indonesia bangsa indonesi...
1       pertama kali pergi pandemi candi borobudur pro...
2       candi borobudur magelang yogyakarta salah reko...
3       baru kali kesini pa sih ajaib dunia sulit baya...
4       candi borobudur wisata kenal dunia ganti sy wi...
                              ...                        
6738    the place great probably everything expect how...
6739    prambanan rara jonggrang javanese romanized ra...
6740    this temple worth price theres much see poorly...
6741    prambanan temple largest hindu temple ancient ...
6742    best choice three mostfamous temple yogyakarta...
Name: text, Length: 6743, dtype: object

## Ekstraksi Fitur
words n-gram. words unigram+bigram+trigram untuk fiturnya dan vektornya diekstraksi dengan CountVectorizer dari pustaka Scikit-Learn

## Eksperimen

### Analisis Sentimen Berbasis Aspek

In [54]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

In [57]:
preprocess = pd.concat([pd.DataFrame(teks), df_clean['daya_tarik'], df_clean['amenitas'], df_clean['aksesibilitas'], df_clean['citra'], df_clean['harga'], df_clean['sdm']], axis=1)
preprocess.head()

,text,daya_tarik,amenitas,aksesibilitas,citra,harga,sdm
0,tinggal sejarah umur indonesia bangsa indonesi...,1,-,-,1,0,0
1,pertama kali pergi pandemi candi borobudur pro...,-,1,-,1,-,1
2,candi borobudur magelang yogyakarta salah reko...,1,-,-,1,-,-
3,baru kali kesini pa sih ajaib dunia sulit baya...,1,-,-,-,-1,-
4,candi borobudur wisata kenal dunia ganti sy wi...,1,1,-,1,-,-


In [28]:
import pandas as pd
# Melakukan melting pada dataframe
melted_df = preprocess.melt(id_vars=['text'], value_vars=['daya_tarik', 'amenitas', 'aksesibilitas', 'citra', 'harga', 'sdm'],
                    var_name='aspect', value_name='sentiment')
# Menghapus baris dengan nilai '-' pada kolom sentiment
melted_df = melted_df[melted_df['sentiment'] != '-']

melted_df.head()

,text,aspect,sentiment
0,tinggal sejarah umur indonesia bangsa indonesi...,daya_tarik,1
2,candi borobudur magelang yogyakarta salah reko...,daya_tarik,1
3,baru kali kesini pa sih ajaib dunia sulit baya...,daya_tarik,1
4,candi borobudur wisata kenal dunia ganti sy wi...,daya_tarik,1
5,wajar salah ajaib dunia candi bangun tingkat t...,daya_tarik,1


In [58]:
# Mengubah nilai sentiment
sentiment_mapping = {1: 'positive', 0: 'neutral', -1: 'negative'}
melted_df['sentiment'] = melted_df['sentiment'].astype(int).map(sentiment_mapping)
melted_df.head()

,text,aspect,sentiment
0,tinggal sejarah umur indonesia bangsa indonesi...,daya_tarik,positive
2,candi borobudur magelang yogyakarta salah reko...,daya_tarik,positive
3,baru kali kesini pa sih ajaib dunia sulit baya...,daya_tarik,positive
4,candi borobudur wisata kenal dunia ganti sy wi...,daya_tarik,positive
5,wajar salah ajaib dunia candi bangun tingkat t...,daya_tarik,positive


In [59]:
# Memisahkan dataframe berdasarkan aspek
dfs_melted_df = {aspect: melted_df[melted_df['aspect'] == aspect] for aspect in melted_df['aspect'].unique()}

In [68]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, cross_val_predict, StratifiedKFold
import warnings
warnings.filterwarnings('ignore', category=UserWarning)

In [66]:
# Daftar algoritma klasifikasi
classifiers = {
    'Naive Bayes': MultinomialNB(),
    'Logistic Regression': LogisticRegression(),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Extra Trees': ExtraTreesClassifier()
}

In [67]:
# Melakukan pelatihan dan evaluasi model untuk setiap aspek dan setiap algoritma
for aspect, df_aspect in dfs_melted_df.items():
    print(f'\nAspect: {aspect}')
    # Membagi data menjadi data latih dan data uji
    X = df_aspect['text']
    y = df_aspect['sentiment']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    for clf_name, clf in classifiers.items():
        # Membuat pipeline dengan TfidfVectorizer dan classifier
        pipeline = Pipeline([
            ('tfidf', TfidfVectorizer(max_features=100000)),
            ('clf', clf)
        ])
        # Melakukan pelatihan model
        pipeline.fit(X_train, y_train)

        # Melakukan prediksi
        y_pred = pipeline.predict(X_test)

        # Evaluasi model
        print(f'\nClassification Report with {clf_name}:')
        print(classification_report(y_test, y_pred))


Aspect: daya_tarik

Classification Report with Naive Bayes:
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00         9
     neutral       0.00      0.00      0.00        64
    positive       0.94      1.00      0.97      1120

    accuracy                           0.94      1193
   macro avg       0.31      0.33      0.32      1193
weighted avg       0.88      0.94      0.91      1193


Classification Report with Logistic Regression:
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00         9
     neutral       0.40      0.03      0.06        64
    positive       0.94      1.00      0.97      1120

    accuracy                           0.94      1193
   macro avg       0.45      0.34      0.34      1193
weighted avg       0.90      0.94      0.91      1193


Classification Report with Decision Tree:
              precision    recall  f1-score   support

    negative       0.09      0

In [69]:
# Melakukan pelatihan dan evaluasi model untuk setiap aspek dan setiap algoritma dengan cross-validation
for aspect, df_aspect in dfs_melted_df.items():
    print(f'\nAspect: {aspect}')
    # Membagi data menjadi fitur (X) dan target (y)
    X = df_aspect['text']
    y = df_aspect['sentiment']
    
    for clf_name, clf in classifiers.items():
        # Membuat pipeline dengan TfidfVectorizer dan classifier
        pipeline = Pipeline([
            ('tfidf', TfidfVectorizer(max_features=100000)),
            ('clf', clf)
        ])
        
        # Menggunakan cross-validation untuk menghasilkan prediksi
        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        y_pred = cross_val_predict(pipeline, X, y, cv=skf)

        # Evaluasi model
        print(f'\nClassification Report with {clf_name}:')
        print(classification_report(y, y_pred))


Aspect: daya_tarik

Classification Report with Naive Bayes:
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00        56
     neutral       0.00      0.00      0.00       345
    positive       0.93      1.00      0.97      5561

    accuracy                           0.93      5962
   macro avg       0.31      0.33      0.32      5962
weighted avg       0.87      0.93      0.90      5962


Classification Report with Logistic Regression:
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00        56
     neutral       0.62      0.01      0.03       345
    positive       0.93      1.00      0.97      5561

    accuracy                           0.93      5962
   macro avg       0.52      0.34      0.33      5962
weighted avg       0.91      0.93      0.90      5962


Classification Report with Decision Tree:
              precision    recall  f1-score   support

    negative       0.04      0

### Pemodelan Topik

## Evaluasi